Load the Model and Label Binarizer

In [1]:
import torch
import pickle
from model.model import EnhancedModel  # adjust import if needed

# Load label binarizer
with open("model//model_labels.dat", "rb") as f:
    lb = pickle.load(f)

# Load model
num_classes = 36
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model = EnhancedModel(num_classes=num_classes).to(device)
model.load_state_dict(torch.load("model//captcha_recognition_model.pth", map_location=device))
model.eval()

EnhancedModel(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): LeakyReLU(negative_slope=0.1)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (res1): ResidualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): LeakyReLU(negative_slope=0.1)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): LeakyReLU(negative_slope=0.1)
  )
  (drop1): Dropout(p=0.2, inplace=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (res2): ResidualBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1)

Prepare the Test DataLoader

In [2]:
from torchvision import transforms
from torch.utils.data import DataLoader
from model.model import CaptchaDataset  # adjust import if needed

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

# Recreate test_samples as in your training script
import os
from sklearn.model_selection import train_test_split

LETTER_IMAGES_FOLDER = "extracted_characters"
all_samples = []
for label in os.listdir(LETTER_IMAGES_FOLDER):
    label_folder = os.path.join(LETTER_IMAGES_FOLDER, label)
    if os.path.isdir(label_folder):
        for image_file in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_file)
            all_samples.append((image_path, label))

_, test_samples = train_test_split(all_samples, test_size=0.25, random_state=42)
test_dataset = CaptchaDataset(test_samples, lb, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

Run Inference and Collect Predictions

In [3]:
import numpy as np

all_preds = []
all_targets = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(torch.sigmoid(outputs), dim=1)
        targets = torch.argmax(labels, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

Compute Precision, Recall, and F1

In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(all_targets, all_preds, average='macro')
recall = recall_score(all_targets, all_preds, average='macro')
f1 = f1_score(all_targets, all_preds, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Precision: 0.9182
Recall:    0.9137
F1 Score:  0.9146
